# Répertoire de voix (Accès restreint)

**IMPORTANT :** Demandez une autorisation sur : https://aka.ms/azure-speaker-recognition. Créez une ressource *Voix* ou *Speech* et renseignez `SERVEUR` avec le serveur de votre point de terminaison (endpoint). Récupérez également la clé et renseignez là lors de l'exécution de la première cellule.

In [1]:
from getpass import getpass

CLE = getpass("Clé")
SERVEUR = 'https://francecentral.api.cognitive.microsoft.com'  # Votre serveur de endpoint

## Création du profil

In [ ]:
import requests

reponse = requests.post(
    url=f'{SERVEUR}/speaker-recognition/identification/text-independent/profiles?api-version=2021-09-05',
    headers={'Ocp-Apim-Subscription-Key': CLE},
    json={'locale': 'fr-FR'}
)
resultat = reponse.json()
resultat

In [ ]:
profil = resultat['profileId']
profil

## Référencement de la voix

In [ ]:
import requests

reponse = requests.post(
  url=f'{SERVEUR}/speaker-recognition/identification/text-independent/profiles/{profil}/enrollments?api-version=2021-09-05&ignoreMinLength=true',
  headers={
    'Ocp-Apim-Subscription-Key': CLE,
    'Content-Type': 'audio/wav'
  },
  data=open('data/FR_1_b.WAV.WAV', 'rb').read()
)
reponse.json()